In [ ]:
import numpy as np 
import nltk
nltk.download('stopwords')
import sys 
from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords 
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils 
from keras.callbacks import ModelCheckpoint


In [8]:
a = "/content/frank.txt"
file = open(a).read()

In [10]:
def tokenize_words(input):
    input = input.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)

processed_inputs = tokenize_words(file)

In [ ]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [ ]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total no. of characters:",input_len)
print("Total vocab:",vocab_len)

Total no. of characters: 232972
Total vocab: 37


In [ ]:
seq_length = 100
x_data = []
y_data = []

In [ ]:
for i in range (0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print("Total patterns:", n_patterns)

Total patterns: 232872


In [ ]:
X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [ ]:
y = np_utils.to_categorical(y_data)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks =[checkpoint]

In [ ]:
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

In [ ]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='data')

In [ ]:
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed: ")
print("\".".join([num_to_char[value] for value in pattern]),"\"")


In [ ]:
for i in range(1000):
    x = np.reshape(pattern,(l,len(pattern), l))
    x = x/float(vocab_len)
    prediction = model.predict(x, verbose = 0)
    index = np.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[l:len(pattern)]
